# SDP Certification of $C_{1a}$ — Negative Results

## Summary
Three SDP relaxation approaches were tested. **All failed** to produce useful bounds.

| Approach | Bound | Failure Mode |
|----------|-------|--------------|
| Fourier SDP (cosine basis + Fejér-Riesz) | $\eta = 2$ for all K | Even-function restriction: $\|f*f\|_\infty = \|f\|_2^2 \geq 2$ |
| Spatial Shor+RLT SDP | $\eta \to 2P/(2P-1) \to 1$ | Relaxation too loose: full-rank $X^*$ spreads mass uniformly |

**This answers K3 negatively: basic SDP relaxations are not tight for this problem.**

## Key insights
- The Shor relaxation replaces $X = xx^T$ with $X \succeq xx^T$, allowing the SDP to spread mass uniformly across all $2P-1$ anti-diagonals — impossible for any rank-1 solution
- The Fourier/cosine basis restricts to even functions, for which $\|f*f\|_\infty = \|f\|_2^2 \geq 2$ always (trivial)
- Higher-order Lasserre hierarchy or Fourier-domain formulations with sine+cosine basis needed (see `lasserre_level2.ipynb`)

In [ ]:
import numpy as np
import cvxpy as cp
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import time

print(f"cvxpy version: {cp.__version__}")

## 1. Fourier-Domain SDP (Cosine Basis)

Represent $f(x) = \sum_{k=0}^K a_k \cos(4\pi k x)$ on $[-1/4, 1/4]$, with $a_0 = 2$ ($\int f = 1$).
Enforce $\eta - \frac{1}{2}\sum_n C_n \cos(2\pi n t) \geq 0$ via Toeplitz PSD (Fejér-Riesz).

**Result**: $\eta = 2$ for all K. The cosine basis forces $f$ to be even, and for even nonneg $f$ with $\|f\|_1=1$: $\|f*f\|_\infty = \|f\|_2^2 \geq 2$.

**Caveat**: The aperiodic autoconvolution of a compactly supported function is NOT a trigonometric polynomial, so Fejér-Riesz is only approximate.

In [ ]:
def solve_fourier_sdp(K, n_nonneg_pts=100):
    """Fourier-domain SDP relaxation using cosine basis."""
    n = K + 2
    Y = cp.Variable((n, n), symmetric=True)
    eta = cp.Variable()
    constraints = [Y >> 0, Y[0, 0] == 1, Y[0, 1] == 2]
    
    C = []
    for nn in range(2 * K + 1):
        terms = [Y[1 + j, 1 + nn + j] for j in range(K + 1) if 0 <= nn + j <= K]
        C.append(cp.sum(terms) if terms else cp.Constant(0.0))
    
    m = 2 * K + 1
    T = cp.Variable((m, m), symmetric=True)
    for i in range(m):
        for j in range(i, m):
            d = j - i
            if d == 0: constraints.append(T[i, j] == eta - 0.5 * C[0])
            elif d < len(C): constraints.append(T[i, j] == -0.5 * C[d])
            else: constraints.append(T[i, j] == 0)
    constraints.append(T >> 0)
    
    x_pts = np.linspace(-0.25, 0.25, n_nonneg_pts + 2)[1:-1]
    for x_i in x_pts:
        cos_vals = [np.cos(4 * np.pi * k * x_i) for k in range(K + 1)]
        constraints.append(cp.sum([cos_vals[k] * Y[0, 1 + k] for k in range(K + 1)]) >= 0)
    
    prob = cp.Problem(cp.Minimize(eta), constraints)
    try:
        prob.solve(solver='SCS', max_iters=100000, eps=1e-8, verbose=False)
    except:
        prob.solve(solver='CLARABEL', verbose=False)
    
    if prob.status in ('infeasible', 'unbounded', None):
        return None, prob.status
    return eta.value, prob.status


print("K  | SDP bound | Status")
print("-" * 35)
for K in [2, 5, 10, 20, 30]:
    eta_val, status = solve_fourier_sdp(K)
    print(f"{K:>2} | {eta_val:.6f}  | {status}" if eta_val else f"{K:>2} | FAILED    | {status}")

print("\n=> All give eta=2.0 (even-function restriction).")

## 2. Spatial-Domain SDP (Shor + RLT)

Discretize $f$ as P uniform bins on $[-1/4, 1/4]$, width $w = 1/(2P)$.
Simplex variables $x_i = h_i \cdot w$. Objective: $\max_k 2P \sum_{i+j=k} x_i x_j$.

SDP lifts $xx^T$ to PSD variable $X \succeq xx^T$ with RLT cuts ($X \geq 0$, $X\mathbf{1} = x$).

**Result**: SDP bound = $2P/(2P-1) \to 1$ for all P tested (2..50).

In [ ]:
def build_antidiag_matrices(P):
    As = []
    for k in range(2 * P - 1):
        A = np.zeros((P, P))
        for i in range(P):
            j = k - i
            if 0 <= j < P: A[i, j] = 1.0
        As.append(A)
    return As


def solve_sdp(P):
    """Shor + RLT SDP relaxation."""
    As = build_antidiag_matrices(P)
    Y = cp.Variable((P + 1, P + 1), symmetric=True)
    eta = cp.Variable()
    x = Y[0, 1:]; X = Y[1:, 1:]
    constraints = [
        Y >> 0, Y[0, 0] == 1,
        cp.sum(x) == 1, x >= 0,
        X >= 0, X @ np.ones(P) == x,
    ]
    for A_k in As:
        constraints.append(eta >= 2 * P * cp.trace(A_k @ X))
    prob = cp.Problem(cp.Minimize(eta), constraints)
    prob.solve(solver='SCS', max_iters=50000, eps=1e-8, verbose=False)
    if prob.status in ('optimal', 'optimal_inaccurate'):
        eigvals = np.linalg.eigvalsh(Y.value[1:, 1:])
        rank = int(np.sum(eigvals > 1e-6 * np.max(eigvals)))
        return eta.value, rank, prob.status
    return None, None, prob.status


def solve_primal(P, n_restarts=20, seed=42):
    """Primal upper bound via softmax + L-BFGS-B."""
    rng = np.random.RandomState(seed)
    def softmax(z):
        z = z - np.max(z); e = np.exp(z); return e / np.sum(e)
    def objective(z):
        x = softmax(z); return np.max(np.convolve(x, x)) * 2 * P
    best = np.inf
    for _ in range(n_restarts):
        res = minimize(objective, rng.randn(P)*0.5, method='L-BFGS-B',
                       options={'maxiter': 2000, 'ftol': 1e-15})
        best = min(best, res.fun)
    return best


print(f"{'P':>4} | {'SDP LB':>10} | {'2P/(2P-1)':>10} | {'Primal UB':>10} | {'Rank':>5}")
print("-" * 55)
for P in [5, 10, 20, 30, 50]:
    sdp_val, rank, status = solve_sdp(P)
    primal = solve_primal(P, n_restarts=10)
    theory = 2 * P / (2 * P - 1)
    if sdp_val:
        print(f"{P:>4} | {sdp_val:>10.6f} | {theory:>10.6f} | {primal:>10.6f} | {rank:>5}")

print("\n=> SDP bound matches 2P/(2P-1) exactly. Rank(X*) = P (maximally loose).")

## 3. Why These SDPs Fail

### Fourier SDP
Cosine basis $\Rightarrow$ even $f$ $\Rightarrow$ $(f*f)(0) = \|f\|_2^2 \geq 2$. Cannot see the non-even $f$ that achieve $C_{1a} \approx 1.50$.

### Spatial Shor+RLT
The $2P-1$ anti-diagonal sums of $X$ total 1 (from $\mathbf{1}^T X \mathbf{1} = 1$). The SDP spreads mass uniformly at $1/(2P-1)$ per anti-diagonal, giving $\eta = 2P/(2P-1)$. No rank-1 solution can achieve this because $\text{conv}(x,x)$ of a simplex vector is necessarily peaked.

### Conclusions
1. **K3 answered negatively**: Shor+RLT SDP is NOT tight. Gap is structural, not numerical.
2. **Lasserre level-2** (see `lasserre_level2.ipynb`) improves over Shor but only at very small P.
3. **Fourier-domain SDP with sine+cosine** basis needed to handle non-even functions.
4. All evidence points to fundamental weakness of first-order (Shor) lifts for min-of-max-quadratic structure.